In [5]:
from __future__ import division
import random

def simulate(n, case=min):
    urns = [1, 1]
    for i in range(2, n):
        if random.randrange(i) < urns[0]:
            urns[0] += 1
        else:
            urns[1] += 1
    return case(urns)

def estimate(n, tries=1000, sim=simulate, case=min):
    return sum(sim(n, case) for _ in range(tries)) / tries


In [19]:
def histogram(n, tries=1000, sim=simulate):
    stats = [0] * ((n+2) // 2)
    for _ in range(tries):
        stats[sim(n)] += 1
    return stats

def distribute(n, tries=1000, sim=simulate):
    return [s / tries for s in histogram(n, tries)]


In [10]:
[estimate(5, tries=10**5) for _ in range(3)]

[1.50048, 1.4962, 1.49741]

In [13]:
%time
[estimate(4, tries=10**5, case=max) for _ in range(3)]

Wall time: 0 ns


[2.67003, 2.66717, 2.66694]

In [16]:
%%time
for i in range(5, 10):
    print(i, [estimate(i, tries=10**5) for _ in range(3)])

5 [1.5019, 1.50234, 1.50049]
6 [1.80572, 1.80082, 1.7966]
7 [1.99922, 2.00135, 1.99977]
8 [2.28481, 2.28678, 2.2826]
9 [2.50554, 2.49643, 2.49904]
Wall time: 10.1 s


In [21]:
%%time
for i in range(3, 20):
    print(i, distribute(i, tries=10**5))

3 [0.0, 1.0]
4 [0.0, 0.66739, 0.33261]
5 [0.0, 0.50165, 0.49835]
6 [0.0, 0.39884, 0.40104, 0.20012]
7 [0.0, 0.33609, 0.33471, 0.3292]
8 [0.0, 0.28628, 0.2864, 0.28516, 0.14216]
9 [0.0, 0.25261, 0.24991, 0.24897, 0.24851]
10 [0.0, 0.21981, 0.22395, 0.22072, 0.22243, 0.11309]
11 [0.0, 0.20194, 0.19858, 0.20128, 0.19807, 0.20013]
12 [0.0, 0.18369, 0.18046, 0.18154, 0.1823, 0.18197, 0.09004]
13 [0.0, 0.16602, 0.16517, 0.1669, 0.16815, 0.16603, 0.16773]
14 [0.0, 0.15387, 0.15315, 0.15472, 0.15422, 0.15396, 0.15287, 0.07721]
15 [0.0, 0.1421, 0.1444, 0.14341, 0.14076, 0.1428, 0.14286, 0.14367]
16 [0.0, 0.13284, 0.13435, 0.13198, 0.13534, 0.13122, 0.13437, 0.13232, 0.06758]
17 [0.0, 0.12444, 0.12528, 0.12529, 0.12533, 0.1249, 0.12558, 0.12513, 0.12405]
18 [0.0, 0.11702, 0.11666, 0.11852, 0.11831, 0.11599, 0.11744, 0.11891, 0.11831, 0.05884]
19 [0.0, 0.11182, 0.11215, 0.11196, 0.11184, 0.1101, 0.1107, 0.11007, 0.11142, 0.10994]
Wall time: 19.9 s


There is a nice pattern over here:

$ 2 \rightarrow [0, 1]$

$ 3 \rightarrow [0, 1]$

What next?

$ 4 \rightarrow [0, \frac{2}{3}, \frac{1}{3}] $

$ 5 \rightarrow [0, \frac{1}{2}, \frac{1}{2}] \rightarrow [0, \frac{2}{4}, \frac{2}{4}] $

In [23]:
def estimate2(n, tries=10**5):
    return sum(n * x for n, x in zip(range(n), distribute(n, tries=tries)))

In [24]:
%%time
for i in range(5, 10):
    print(i, estimate(i, tries=10**5))

5 1.50099
6 1.80036
7 2.00139
8 2.28547
9 2.50619
Wall time: 3.49 s


$ \frac{2}{N - 1} \cdot \sum_{i = 1}^{\lfloor \frac{N}{2} \rfloor} i $

In [25]:
def calculate(n):
    if n % 2 == 1:
        max_coins = n // 2 # floor
        # s = (max_coins + 1) * max_coins / 2
        # return 2 * s / (n - 1)
        return (max_coins + 1) * max_coins / (n - 1)
    max_coins = n // 2 # equal
    s = (max_coins) * (max_coins-1) / 2
    return 2 * s / (n-1) + max_coins / (n - 1)

        

In [26]:
%%time
for i in range(5, 10):
    print(i, calculate(i))

5 1.5
6 1.7999999999999998
7 2.0
8 2.2857142857142856
9 2.5
Wall time: 1 ms


In [27]:
calculate(10**6)

250000.25000025

As you all see, the expected value of coins in the emptier urn is about 250,000.25 &mdash; how much should one pay for that? I think it's up to you, I would be happy to get this urn for free.

This is noticeably close to $\frac{1}{4} N$, which should not be so surprising: this is the limit in $\infty$.

Except for this pattern that I discovered, I can't intuitively explain why this is so.

### Closed formula, without if

Not sure yet, if it is so close at hand, but:

$ M_c = \lfloor \frac{N}{2} \rfloor $

In [8]:
def calculate2(n):
    max_coins = n // 2
    return (max_coins * (max_coins - 1) + max_coins * (1 + n % 2)) / (n - 1)

In [9]:
calculate2(10**6)

250000.25000025

In [10]:
def calculate3(n):
    max_coins = n // 2
    return max_coins * (max_coins + n % 2) / (n - 1)

In [12]:
[calculate3(n) for n in [10**6, 3, 4, 5]]

[250000.25000025, 1.0, 1.3333333333333333, 1.5]

In [15]:
[(n, n // 2, n // 2 + n % 2, (n+1) // 2) for n in range(10)]

[(0, 0, 0, 0),
 (1, 0, 1, 1),
 (2, 1, 1, 1),
 (3, 1, 2, 2),
 (4, 2, 2, 2),
 (5, 2, 3, 3),
 (6, 3, 3, 3),
 (7, 3, 4, 4),
 (8, 4, 4, 4),
 (9, 4, 5, 5)]

In [17]:
def calculate4(n):
    return (n // 2) * ((n + 1) // 2) / (n - 1)

def calculate4f(n):
    return (n / 2) * ((n + 1) / 2) / (n - 1)


In [36]:
import numpy as np
from itertools import chain
[(n, calculate4(n), calculate4f(n)) # , calculate4(n) /  calculate4f(n)) 
          for n in chain(range(3, 21), [100, 1000, 10**4, 10**6 - 1, 10**6])]

[(3, 1.0, 1.5),
 (4, 1.3333333333333333, 1.6666666666666667),
 (5, 1.5, 1.875),
 (6, 1.8, 2.1),
 (7, 2.0, 2.3333333333333335),
 (8, 2.2857142857142856, 2.5714285714285716),
 (9, 2.5, 2.8125),
 (10, 2.7777777777777777, 3.0555555555555554),
 (11, 3.0, 3.3),
 (12, 3.272727272727273, 3.5454545454545454),
 (13, 3.5, 3.7916666666666665),
 (14, 3.769230769230769, 4.038461538461538),
 (15, 4.0, 4.285714285714286),
 (16, 4.266666666666667, 4.533333333333333),
 (17, 4.5, 4.78125),
 (18, 4.764705882352941, 5.029411764705882),
 (19, 5.0, 5.277777777777778),
 (20, 5.2631578947368425, 5.526315789473684),
 (100, 25.252525252525253, 25.505050505050505),
 (1000, 250.25025025025025, 250.5005005005005),
 (10000, 2500.2500250025, 2500.5000500050005),
 (999999, 250000.0, 250000.2500005),
 (1000000, 250000.25000025, 250000.5000005)]

So, there is our neat closed-from expression for the estimated number of coins in the smaller urn:

$ E_s(N) = \frac{\lfloor \frac{N}{2} \rfloor \cdot \lfloor \frac{N+1}{2} \rfloor}{N - 1} $